In [ ]:
from htmd import *
from htmd.molecule.util import maxDistance, writeVoxels
htmd.config(viewer='vmd')

In [ ]:
sims = simlist(glob('/pub.htmd.org/CXCL12-confAnalysis/*/'), glob('/pub.htmd.org/CXCL12-confAnalysis/*/structure.pdb'))

In [ ]:
names_sys = list()

for sim in range(1,len(sims)+1):
    names_sys.append("sys"+str(sim))
    

Keep with different names the elements (simulations) of the simlist into another list.

In [ ]:
count = 0
maxlist=list()
for sim in sims:
    names_sys[count]= Molecule(sim)
    names_sys[count].wrap('protein')
    names_sys[count].align('protein')
    maxlist.append(maxDistance(names_sys[count], 'all'))
    count+=1

Keep each of the simulations as a Molecule object and wrap and align them to move the molecules into the simulation box and to fix them. At the same time, computes the maximum distance of the protein to the farthest water molecule of the first frame of each simulation. 

In [ ]:
D=int(round(max(maxlist)))
D

From the list of the maximum distances it take the maximum ("the max of maxs") into variable D. This is important as,it can be used as a reference to move the protein to the positive quadrant/values in the space (see below)

In [ ]:
mat_scores = np.zeros((2*D+15,2*D+15,2*D+15))

Because the "water box" moves around the fixed protein describing an sphere, the matrix of the water scores (mat_scores) should be two times the maximum distance plus some margin (15 units) just in case. 

In [ ]:
for sys in names_sys:
    sys.moveBy([D,D,D])
    wat=sys.copy()
    wat.filter('name OH2')
    mask=(wat.coords<D*2+15) & (wat.coords>-D*2-15)
    wat2 = wat.coords*mask
    for atom in range(0, wat2.shape[0]): 
        for frame in range(0, wat2.shape[2]):
            x = int(round(wat2[atom][0][frame]))
            y = int(round(wat2[atom][1][frame]))
            z = int(round(wat2[atom][2][frame]))
            mat_scores[x][y][z] += 1

It moves the simulation according to the maximum distance and makes a copy to filter out only the waters. Because there are some values that go out of the mat_scores boundries (due to some artifact in the dataset) a mask matrix of the same dimensions with boolan values turned to be a good solution to filter out those values that were considered "False"(each "False" value turns to 0). 
The rounded numbers of the coordinates of the atoms are used to fill the matrix scores. This way avoids to iterate through the 3 space axis (x,y,z) that would have divided the space in little boxes of 1 angstrom adding more loops and therefore more computational cost.


In [ ]:
mat_scores=mat_scores/len(names_sys)

In [ ]:
mat_scores = mat_scores/(wat.coords.shape[0]*wat.coords.shape[2]) 

Mean and normalization of the scores to have the probability matrix. 

In [ ]:
kB = 0.001987191
T = 298

mat_scores= mat_scores+10**(-30)
mat_pot =-np.log(mat_scores)*kB*T

In order to avoid "inf" elements in the matrix potentials (mat_pot) everytime there is a 0 in the matrix scores, let's add 10^-30 to each probability. 


In [ ]:
min_vec=np.array([0,0,0])
max_vec=np.array([2*D+15,2*D+15,2*D+15])
res_vec=np.array([1,1,1])

writeVoxels(mat_pot,'output.cube',min_vec,max_vec,res_vec)

writeVoxel is a function that allows to output the results into a cube file so that we can visualize it in VMD. As the systems have been moved, the minimum vector used is [0,0,0] and the maximum is the same number used to initialize the score matrix.


In [ ]:
names_sys[1].view()